In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory

import tensorflow as tf
import tensorflow_datasets as tfds

Defining image shape

In [2]:
img_width = 512
img_height = 384
colors = 3

Connecting Google Collab with Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Creating datasets from trashnet_dataset_resized

In [6]:
ds_train = image_dataset_from_directory(
    '/content/gdrive/My Drive/trashnet_dataset_resized',
    labels='inferred',
    label_mode = 'categorical',
    batch_size=16,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training',
)

Found 2537 files belonging to 6 classes.
Using 1776 files for training.


In [ ]:
for image, label in ds_train.take(2):
  print(image)
  print(label)

In [7]:
ds_test = image_dataset_from_directory(
    '/content/gdrive/My Drive/trashnet_dataset_resized',
    labels='inferred',
    label_mode = 'categorical',
    batch_size=16,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='validation',
)

Found 2537 files belonging to 6 classes.
Using 761 files for validation.


Preprocessing datasets

In [17]:
def preprocess(image,label):
  image = image / 255.
  return image, label

In [18]:
ds_train = ds_train.map(preprocess)

In [21]:
ds_test = ds_test.map(preprocess)

Defining CNN model

In [22]:
def initialize_model():
  
  model = Sequential()

  model.add(layers.Conv2D(16, (3,3), input_shape=(img_height, img_width, colors), activation='relu'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))

  model.add(layers.Conv2D(16, (4,4), activation='relu'))
  model.add(layers.MaxPool2D(pool_size=(2,2)))

  model.add(layers.Flatten())

  model.add(layers.Dense(6, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

Initializing and fitting model

In [23]:
model = initialize_model()

In [24]:
es = EarlyStopping(patience=15, restore_best_weights=True)

In [25]:
model.fit(ds_train, validation_data = ds_test, epochs=1000, callbacks=[es])

Epoch 1/1000
111/111 [==============================] - 10s 84ms/step - loss: 2.4495 - accuracy: 0.2421 - val_loss: 1.7075 - val_accuracy: 0.2589
Epoch 2/1000
111/111 [==============================] - 9s 82ms/step - loss: 1.6482 - accuracy: 0.3255 - val_loss: 1.6762 - val_accuracy: 0.2917
Epoch 3/1000
111/111 [==============================] - 10s 83ms/step - loss: 1.4530 - accuracy: 0.4454 - val_loss: 1.6068 - val_accuracy: 0.3311
Epoch 4/1000
111/111 [==============================] - 9s 83ms/step - loss: 1.0264 - accuracy: 0.6340 - val_loss: 1.7259 - val_accuracy: 0.3272
Epoch 5/1000
111/111 [==============================] - 9s 83ms/step - loss: 0.6600 - accuracy: 0.7810 - val_loss: 1.8547 - val_accuracy: 0.3311
Epoch 6/1000
111/111 [==============================] - 9s 83ms/step - loss: 0.4626 - accuracy: 0.8497 - val_loss: 2.4396 - val_accuracy: 0.3522
Epoch 7/1000
111/111 [==============================] - 9s 82ms/step - loss: 0.2651 - accuracy: 0.9217 - val_loss: 2.8245 - val_

Evaluating model

In [27]:
# model.evaluate(ds_test) utiliser le "futur" ds_test